# Exploring covariance between properties across scales
**Authors**: Benjamin Jasperson, Harley T. Johnson

**GitHub URL**: https://github.com/bjasperson/property-covariance-demo

# Background Information

This workbook explores the use of interatomic potential (IP)-generated data to uncover correlations between fundamental microscopic properties, which we call canonical properties, and large-scale quantities of interest (QoI).
It closely follows the work outlined in [this manuscript](https://doi.org/10.1016/j.actamat.2025.120722).

As a proof-of-principle, we'll use grain boundary (GB) energy as our QoI. 
Specifically, we'll look at how canonical properties relate to the scaling factor in the universal lattice matching (LM) model of Runnels et al. (2016).
You don't need to worry about how this coefficient is defined; see the paper for details. Just think of it as a scalar measure of grain boundary energy.

By the end of this workbook, you will have:
1. Explored a dataset of IP-generated property results, uncovering correlations that can be used in a multiscale regression model.
2. Developed a regression model using canonical properties identified through k-fold cross-validation.
3. Used the regression model with first-principles indicator properties to make a prediction for the GB scaling coefficient from first-principles canonical properties.

In [21]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, RepeatedKFold, cross_val_score, LeaveOneOut, GridSearchCV
from sklearn.impute import KNNImputer
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model, svm

from itertools import combinations
from textwrap import wrap

# Download Data
To begin, we need to import our project data. The data we will use is originally from [this manuscript](https://arxiv.org/abs/2411.16770), [published here](https://doi.org/10.1016/j.actamat.2025.120722). You can find the original paper repo [here](https://github.com/Johnson-Research-Group/gb_covariance).

The data includes IP-generated canonical properties, along with an LM model scaling factor calculated from individual symmetric-tilt GB energy simulations.

**Task**: Using pandas, download the CSV file with this [link](https://github.com/bjasperson/property-covariance-demo/blob/main/data/gb_data.csv?raw=true), save the dataframe as `df_data`, and look at the first five rows.

In [ ]:
url_link = "https://github.com/bjasperson/property-covariance-demo/blob/main/data/gb_data.csv?raw=true"

# Data review
It is important to get a feel for the data that you are working with. To make plotting a little nicer, we have a csv file with key/value pairs to convert the variable names into human readable format.  

**Task**: import the label csv file, located at `./data/label_dict.csv`, and convert it to a dictionary called `label_dict` using pandas, for use with plotting.

In [23]:
def import_label_dict(label_dict_path = "https://github.com/bjasperson/property-covariance-demo/blob/main/data/label_dict.csv?raw=true"):
    """import the ./data/label_dict.csv file, convert it to a dictionary

    Args:
        label_dict_path (str, optional): path to csv file with key/value pairs.

    Returns:
        dict: a dictionary with df column names as keys, and human-readable label as value
    """
    
    return label_dict

label_dict = import_label_dict()

Now, let's write a function that calculates the [correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) between two variables. 

**Task**: given our dataframe, create a function that uses [Pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html) to calculate the correlation coefficient between each set of variables. First, make a copy of the dataframe and replace the column names with the human readable version from the label dictionary using list comprehension. Next, apply the `corr` method to the copied dataframe to create `df_corr`. Finally, sort both the columns *and* rows of `df_corr` based on how correlated the variable is to the scaling factor, `$E_0$`.

In [24]:
def correlation_df(df, label_dict):
    """create df with correlation values

    Args:
        df (pandas.DataFrame): property data (columns) for a given IP (row) 
        label_dict (dict): the label dict from above, to convert label keys to human readable format

    Returns:
        pandas.DataFrame: correlation coefficient between canonical properties and coeff ($E_0$). Sort both the columns and rows by how correlated they are with the coeff $E_0$.
    """

    return df_corr

We now want to look at how the different fundamental properties relate with each other and our Quantity of Interest (GB energy coefficient). To aid in this, we will look at two plots.

First, let's plot a heat map of the correlation coefficient. 

In [ ]:
def corr_plot(df, 
              label_list, 
              label_dict, 
              annot = False, 
              figsize = (10,10),
              annotation_fontsize = 6,
              tick_fontsize = 6,
              ):
    
    df_corr = correlation_df(df[label_list], label_dict)
    fig,ax = plt.subplots(figsize = (figsize[0],figsize[1]))
    colors = sns.color_palette("vlag", as_cmap=True)
    sns.heatmap(df_corr, vmin = -1, vmax = 1, cmap=colors,
                annot=annot,
                xticklabels=True,
                yticklabels=True,
                fmt = '.1f',
                annot_kws = {"fontsize":annotation_fontsize},
                cbar_kws = {"location":"top"}
                )
    ax.tick_params(labelsize=tick_fontsize)
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=tick_fontsize)

label_list = df_data.columns.to_list()
label_list.remove("crystal_type")
label_list.remove("species")
label_list.remove("model")
corr_plot(df_data, label_list, label_dict)

Notice how properties that are highly correlated with each other appear as square blocks in the plot. This makes it easy to identify groups of properties that can easily be substituted for each other other in the model. 

Next, let's look at pairplots of the most correlated features. 

**Task**: create a function to plot a set of properties from the property dataframe. Use Seaborn pairplot, and set the marker color (`hue`) based on `species`. Explore different combinations of pairplots for the data.

In [ ]:
def pairplot_fun(df, 
                 params_list,  
                 label_dict, 
                 height=1.5,
                 ):
    """create pairplot for select indicator properties

    Args:
        df (pandas.DataFrame): dataframe of data that was previously imported
        params_list (list): list of parameter strings to use
        label_dict (dict): the label dict from above, to convert label keys to human readable format
        height (float): pairplot height. Defaults to 2.0.
        xlims (list): list of limits to use for plotting
    """

    return


plot_param_list = ["coeff", "lattice_constant_fcc", "lattice_constant_bcc", 'intr_stack_fault_energy_fcc', 'unstable_stack_energy_fcc']
pairplot_fun(df_data, plot_param_list, label_dict)

Some questions to ponder:
1. Did you find factors that you think are worth including?
2. Do you see any outliers that we need to remove? How will you decide what is an outlier?

# Identifying indicator properties

Now that we have our data, we want to figure out which canonical properties will be best for our model. To do that, we'll use a [linear regression model](https://scikit-learn.org/stable/modules/linear_model.html#ordinary-least-squares) with [k-fold cross-validation](https://scikit-learn.org/stable/modules/cross_validation.html#k-fold).

## Develop regression model

**Task**: Write a function that returns a linear regression pipeline. Your pipeline should include the following steps:
- StandardScaler: for scaling the input values; helpful for SVR, used in LR for consistency. 
- KNNImputer: for filling in the missing property values in our data. Set `n_neighbors=2` and `keep_empty_features=True`
- LinearRegression: the linear regression model (`linear_model.LinearRegression()`).

Combine these in a `Pipeline` to facilitate easy use .

In [27]:
def linear_regr_pipeline():
    """creates a linear regression pipeline for modeling purposes

    Returns:
        sklearn.pipeline.Pipeline: linear regression pipeline
    """

    return pipe

pipe = linear_regr_pipeline()

Now, we need to make a list of the properties we are interested in considering as indicator properties for our model. A starting point is provided below.

**If time**: explore different properties, and see if any new additions result in improved models!

In [28]:
# list of parameters to explore
params_list = ['lattice_constant_fcc',
                'bulk_modulus_fcc', 'c11_fcc', 'c12_fcc', 'c44_fcc',
                'extr_stack_fault_energy_fcc', 'intr_stack_fault_energy_fcc', 'unstable_stack_energy_fcc'
                ]

Using the parameter list, we want to make a list of possible combinations up to a certain number of factors. 

**Task**: write a function that uses the `combinations` function from `itertools` to make a list of lists, with possible combinations of factors up to `n_factor_max`.

In [ ]:
def create_subsets_list(factor_list, n_factor_max):
    """create all possible combinations of factors from a list, up to n_factor_max

    Args:
        factor_list (list): a list of individual factors (str) to consider
        n_factor_max (int): maximum number of factors to consider in one model

    Returns:
        list: a list of lists, with each sub-list containing the factors (str) to include (e.g., ['c11_fcc', 'c12_fcc'])
    """

    return subsets

subsets = create_subsets_list(params_list, n_factor_max = 2)
print(subsets)

We now have our list of possible factor combinations to consider. If we fit our model to the entire dataset, we aren't getting a realistic picture of the test error. So, we will use the `cross_val_score` package from scikit-learn to perform cross validation. Read more about it [here](https://scikit-learn.org/stable/modules/cross_validation.html).

**Task**: write a function that iterates over each subset of the possible factor combinations. For each subset, use the `cross_val_score` package to perform cross-validation and return a score. 

In [30]:
def factor_select_cv(df, pipe, subsets, label="coeff", cv=None, scoring='r2'):
    """evaluates combinations of factors
    
    return a dataframe that includes one row per factor combination. list of parameters w/ cv score

    Args:
        df (pandas.DataFrame): dataframe of data that was previously imported
        pipe (sklearn.pipeline.Pipeline): sklearn pipeline
        subsets (list): a list of lists, with each sub-list containing the factors (str) to include (e.g., ['c11_fcc', 'c12_fcc'])
        label (str, optional): the QoI label to use. Defaults to "coeff".
        cv (int, optional): int, cross-validation generator or an iterable, Determines the cross-validation splitting strategy. Defaults to None.
        scoring (str, optional): A str (see model evaluation documentation) or a scorer callable object / function with signature scorer(estimator, X, y) which should return only a single value. Defaults to 'r2'.

    Returns:
        pandas.DataFrame: a dataframe of CV results
    """
    

    return df_results

Now, let's run k-fold CV and get the results.

In [ ]:
# pipe = TransformedTargetRegressor(regressor=pipe,
#                                   transformer=StandardScaler())
filename = "kfold_models_lr"

cv = RepeatedKFold(n_splits=10, n_repeats=5, random_state = 12345)
df_results = factor_select_cv(df_data,
                              pipe, 
                              subsets, 
                              cv=cv, 
                              scoring='neg_root_mean_squared_error')

**If time**: repeat the steps above using different modeling approaches, e.g., support vector regression. Compare performance of the models, and try to understand why some perform better than others.

Sort the values based on "cv_score", with `ascending=False`. 

Pick the top performing model and make a regression model to use. Make predictions and compare predicted versus actual. 

**Task**: fit a pipe against our IP data, using the best two-factor model. Make a prediction using the pipe, and add it to our `df_data` dataframe as "coeff_pred".

Finally, let's look at our results.

**Task**: write a function that takes in the dataframe, plots the actual coefficient on the horizontal axis, and plots the predicted coefficient on the vertical axis. Use seaborn scatterplot with "species" as the hue. Set the aspect ratio to "equal" for each axis, and plot a line along the diagonal to represent a perfect prediction. 

In [35]:
def pred_vs_actual_plot(df, 
                        figsize = (5,5),
                        ):
    sns.set_style("whitegrid")

    return
  

# Make predictions from first-principles indicator properties

Now, we will import the DFT data that we will use with our model to make inferred predictions based on the indicator properties. First, get the data.

**Task**: Using pandas, download the CSV file with this [link](https://github.com/bjasperson/property-covariance-demo/blob/main/data/gb_dft.csv?raw=true), save the dataframe as `df_dft`, and look at the first five rows.

In [ ]:
url_link = "https://github.com/bjasperson/property-covariance-demo/blob/main/data/gb_dft.csv?raw=true"


We have two sets of properties now to consider: one for the IP data, and one for the DFT data. We want to decide which model to use for the DFT data. Let's filter our k-fold CV results to only include the DFT canonical properties.

**Task:** Write a function that filters your model performance list from above based on the DFT properties available. Take in the cv results dataframe and a list of DFT properties (from the df_dft columns). If the properties for a given row are all available with DFT results, include it ([hint](https://www.geeksforgeeks.org/python-check-if-the-list-contains-elements-of-another-list/)). Return a dataframe of new, filtered results. Take a look at the top 5 results.

In [ ]:
def filter_model_results(df, dft_list):
    """takes in a df of k-fold CV results, and filters based on a list of properties

    Args:
        df (pandas.DataFrame): the k-fold CV df from above
        dft_list (list): list of properties available from df_dft

    Returns:
        pandas.DataFrame: filtered results from k-fold CV df
    """


    return 



**Task:** Make a list that identifies the factors from the top performing model. Using your previously created function `add_pred`, make a prediction from DFT indicator properties and add the results to `df_dft` under the name "dft_pred_coeff".

In [ ]:
# best model of the 5 available DFT properties
dft_model_properties = ['c44_fcc',  
                        'unstable_stack_energy_fcc'
                       ]


Now, use the plotting function provided below to show a boxplot of coefficient results, along with the predicted versus actual DFT coefficient.

In [ ]:
def get_boxplot(df_ip, 
                df_dft, 
                plot_errorbar = True,
                order_list = ["Ag","Al","Au","Cu","Ni","Pd","Pt"]):
    """plot boxplot of dft GB results
    """
    sns.set_theme(style="whitegrid")
    fig, ax = plt.subplots(figsize=(4,3))
    sns.boxplot(data = df_ip, 
                x="species", 
                y="coeff", 
                order=order_list, 
                color = "0.8", 
                linewidth=1.0,
                fliersize=5.0,
                whis=0,
                flierprops={"marker":"."},
                zorder=1)
    ax.set_ylabel("GB scaling coefficient")

    # add dft Xs
    ax.scatter(df_dft['species'],
               df_dft['dft_pred_coeff'], 
               marker='x', 
               s=100., 
               alpha=1.0, 
               color="r",
               label='\n'.join(wrap(r"$E_0$ regression prediction using DFT indicator properties",20)),
               zorder=3)

    df_dft_gb_plot = df_dft[['species','dft_exact_coeff']].drop_duplicates()
    df_dft_gb_plot = df_dft_gb_plot[df_dft_gb_plot['species'].isin(order_list)]
    ax.scatter(df_dft_gb_plot['species'],
               df_dft_gb_plot['dft_exact_coeff'], 
               marker='<', 
               s=50., 
               alpha=0.9, 
               color="r",
               label='\n'.join(wrap(r"$E_0$ fit directly to DFT GB results",20)),
               zorder=2)

    # add errorbars if desired
    if plot_errorbar == True:
        ax.errorbar(df_dft['species'],
                    df_dft['regr_coeff'], 
                    yerr = (df_dft['regr_coeff_lower'],
                            df_dft['regr_coeff_upper']), 
                            fmt='.', 
                            markersize=0.0001, 
                            alpha=0.5, 
                            color="r",
                            #label='\n'.join(wrap("Predicted strength using DFT indicator properties",20)),
                            elinewidth=2.0,
                            capsize = 4)

    fig.legend(bbox_to_anchor = (0.05,0.9,0.85,.15),#(0.,1.02,1.,.102),
                    loc='lower left',
                    mode="expand",
                    ncol = 2,
                    fontsize= 8)

    return 

get_boxplot(df_data, df_dft, plot_errorbar=False)

That's it! We see that our model predictions match the DFT results pretty well. 

Additional areas to explore, as time permits:
- Right now, we are using all IP data to fit our model against, then make a prediction. To better estimate the predictive power of our model, we should reserve some data as test data. It makes sense to use cross-validation and report the average error, given our small dataset. Add in cross-validation to make it a more realistic prediction.
- Using different combinations of factors, try to find a better model!
- Explore different model types other than linear regression: SVR, ensemble of regressors, ...
    - see [here](https://scikit-learn.org/stable/machine_learning_map.html) for ideas
- Repeat the process, this time with the strength data
    - [URL link to data](https://github.com/bjasperson/property-covariance-demo/blob/main/data/strength_data.csv?raw=true)